In [11]:
#Set up
from bs4 import BeautifulSoup as bts
import pandas as pd
import requests
import math #for rounding numbers

In [12]:
#Function for parsing the URLs
def getAndParseURL(url): 
    result = requests.get(url, headers = {"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [13]:
#DE - Woman sportwear
url = 'https://www.decathlon.de/browse/c0-damen/c1-sportbekleidung-damen/_/N-qjh2ro'

de1 = getAndParseURL(url)

prod = de1.find_all(class_="dpb-holder loaded svelte-wmr4s2")

In [14]:
#Pagination
url = 'https://www.decathlon.de/browse/c0-damen/c1-sportbekleidung-damen/_/N-qjh2ro'

url_list = [url]
#Getting total products in the category
total_prod = de1.find("div",{"class":"plp-bar-info svelte-1uqvrhu"}).find("span", {"class":"svelte-1uqvrhu"}).text

#No of pages (rounding up all numbers)
math.ceil(int(total_prod)/40)

for i in range(1, math.ceil(int(total_prod)/40)):
        page = f'{url}?from={40 * i}&size=40'
        url_list.append(page)

In [16]:
de_base = "https://www.decathlon.de"
de1_list = []

for url in url_list:
    de1 = getAndParseURL(url)
    prod = de1.find_all(class_="dpb-holder loaded svelte-wmr4s2")
    for product in prod:
        link = product.find("a").attrs["href"]
        prod_cat = de1.find("h1").text
        prod_title = product.find("h2",{"class":"svelte-wmr4s2"}).text
        brand_name = product.find("strong",{"class":"svelte-wmr4s2"}).text
        prod_url = f'{de_base}{link}'

        #if all sku contains only 7 characters: prod_id = product.find("a").attrs["href"].partition("?mc=")[2][0:7]
        #taking sku's even in case of more than 7 character id's:

        if link.partition("?mc=")[2].rpartition('&')[0] == '':
            prod_id = link.partition("?mc=")[2]
        else:
            prod_id = link.partition("?mc=")[2].rpartition('&')[0]

        if product.find("div", {"class":"prc__active-price svelte-1kkqb6z"}) == None:
            reg_price = product.find("span", {"class":"prc__previous"}).text.replace('\n','').replace('*','')
            act_price = product.find("div", {"class":"prc__active-price svelte-1kkqb6z prc__active-price--sale"}).text.replace('\n','').replace('*','')
        else:
            reg_price = product.find("div", {"class":"prc__active-price svelte-1kkqb6z"}).text.replace('\n','').replace('*','')
            act_price = None

        if product.find("div", {"class": "sticker svelte-15lojui"}) == None:
            prod_sticker = None
        else:
            prod_sticker = product.find("div", {"class": "sticker svelte-15lojui"}).text.replace('\n','').replace('*','').replace(' ','')

        de1_list.append({'title': prod_title,
                         'sku': prod_id,
                         'regular price': reg_price,
                         'actual price' : act_price,
                         'brand': brand_name,
                         'url' : prod_url,
                         'sticker' : prod_sticker,
                         'category' : prod_cat})

In [26]:
len(de1_list)

total_prod

de1_list[17]

{'title': 'Workout Ready High-Rise Shorts',
 'sku': '5a0098a8-00f9-4b96-82c1-b56df5145310_c1',
 'regular price': '24,95€',
 'actual price': None,
 'brand': 'REEBOK',
 'url': 'https://www.decathlon.de/p/mp/reebok/workout-ready-high-rise-shorts/_/R-p-5a0098a8-00f9-4b96-82c1-b56df5145310?mc=5a0098a8-00f9-4b96-82c1-b56df5145310_c1&c=SCHWARZ',
 'sticker': None,
 'category': 'Sportbekleidung Damen'}

In [29]:
#Saving into csv:

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import csv
import time

country = 'DE'
fetch_time=f'{datetime.today()}'.replace(":","_")
filename = f'{country}_{prod_cat}_{total_prod}_{fetch_time}.csv'.replace(" ","_")

with open(filename, "w", encoding='utf-8-sig', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter = ",")
    writer.writerow(["title", "sku", "reg_price", "act_price", "brand", "sticker", "category", "url"])

    for item in de1_list:
        writer.writerow([item['title'], item['sku'], item['regular price'], item['actual price'], item['brand'], item['sticker'], item['category'], item['url']])

pd.read_csv(filename, delimiter= ",", nrows=20)



,title,sku,reg_price,act_price,brand,sticker,category,url
0,T-Shirt Fitness Adidas Linear 100 % Baumwolle ...,8624220,"19,99€",NaN,ADIDAS,NaN,Sportbekleidung Damen,https://www.decathlon.de/p/t-shirt-fitness-adi...
1,Strumpfhose Tanzleggings ohne Fuß Mädchen schw...,8608538,"7,99€",NaN,STAREVER,NEU,Sportbekleidung Damen,https://www.decathlon.de/p/strumpfhose-tanzleg...
2,Wanderhose Bergwandern MH500 Damen schwarz,8493683,"29,99€",NaN,QUECHUA,NaN,Sportbekleidung Damen,https://www.decathlon.de/p/wanderhose-bergwand...
3,Wandershorts Naturwandern NH500 Regular Damen ...,8582433,"14,99€","9,99€",QUECHUA,ECODESIGN,Sportbekleidung Damen,https://www.decathlon.de/p/wandershorts-naturw...
4,Laufshirt Run Dry kurzarm atmungsaktiv Damen w...,8543970,"4,99€","3,99€",KALENJI,ECODESIGN,Sportbekleidung Damen,https://www.decathlon.de/p/laufshirt-kurzarm-r...
5,Sport-BH Bustier Basic starker Halt,8563080,"4,99€",NaN,KALENJI,NaN,Sportbekleidung Damen,https://www.decathlon.de/p/sport-bh-bustier-ba...
6,Lauftop atmungsaktiv Kiprun Care Damen grau/blau,8604399,"9,99€","4,99€",KIPRUN,%Endspurt%,Sportbekleidung Damen,https://www.decathlon.de/p/lauftop-kiprun-care...
7,T-Shirt Slim Fitness,8380104,"4,99€",NaN,DOMYOS,NaN,Sportbekleidung Damen,https://www.decathlon.de/p/t-shirt-slim-fitnes...
8,Fleeceweste Bergwandern MH120 Damen schwarz,8072128,"11,99€",NaN,QUECHUA,ECODESIGN,Sportbekleidung Damen,https://www.decathlon.de/p/fleeceweste-bergwan...
9,Laufshirt langarm atmungsaktiv Kiprun Care Dam...,8665686,"19,99€",NaN,KIPRUN,NaN,Sportbekleidung Damen,https://www.decathlon.de/p/laufshirt-langarm-k...


## To-do
1. DONE: Remove the space AND * in price, SPACE in Sticker
2. DONE: Saving URL as a list --> Scrape SKU in each URL OR: trying to extract SKU from the URL
3. DONE: Pagination
4. DONE: Saving as csv. with country_category as file name
5. Wrap chunks of codes up into functions
6. Auto translate category? https://medium.com/analytics-vidhya/how-to-translate-text-with-python-9d203139dcf5
7. Make a list of different categories and countries
8. Loop over cat & countries